In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.1 MB/s eta 0:00:00


# Translation Models


Machine translation is a pivotal field within natural language processing (NLP) that focuses on automating the conversion of text or speech from one language to another. It relies on sophisticated models and techniques to accomplish this challenging task effectively. One of the cornerstone methods in machine translation is the sequence-to-sequence (seq2seq) model, which employs deep neural networks to encode input text and then decode it into the target language. This technique has revolutionized translation tasks by learning to capture complex linguistic nuances and contextual information. Additionally, other models like Transformer-based models, including the famous BERT and GPT-3, have also made significant strides in translation, leveraging attention mechanisms to excel in various language pairs and domains. The choice of model depends on specific translation requirements, language pairs, and the quality of available training data. In this Colab file, we havee given a basic demo on how tto use the dataset and work on a simple seq2seq moel usig RNN.Your task will be to improve the model to the maximum you can ,make prediction on the test dataset given and write a code to generate the BLEU score of you prediction compared to original.






#1. Importing Libraries

In [3]:
import pandas as pd
import numpy as np
#from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional,LSTM, Dropout,Attention
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
import re

#2. Loading and pre-processing the data

### A) Loading the dataset

In [ ]:
#Loading and processing data
eng_fr = pd.read_csv("/content/nlp_intel_train.csv")
ng_fr_test = pd.read_csv("/content/nlp_intel_test.csv")

### B) Dropping NaN texts

In [ ]:
eng_fr = eng_fr.dropna(axis=0, how="any", subset=None, inplace=False)
eng_fr_test = eng_fr_test.dropna(axis=0, how="any", subset=None, inplace=False)

### C) Removing non-alphabetic characters

In [ ]:
eng_fr["en"]=eng_fr["en"].apply(lambda x: x.lower())
eng_fr["fr"]=eng_fr["fr"].apply(lambda x: x.lower())

eng_fr["en"]=eng_fr["en"].apply(lambda x: re.sub("'","",x))
eng_fr["fr"]=eng_fr["fr"].apply(lambda x: re.sub("'","",x))

exclude=set(string.punctuation)
eng_fr["en"]=eng_fr["en"].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
eng_fr["fr"]=eng_fr["fr"].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

eng_fr["en"]=eng_fr["en"].apply(lambda x: re.sub("[1234567890]","",x))
eng_fr["fr"]=eng_fr["fr"].apply(lambda x: re.sub("[1234567890]","",x))

eng_fr["en"]=eng_fr["en"].apply(lambda x: x.strip())
eng_fr["fr"]=eng_fr["fr"].apply(lambda x: x.strip())

eng_fr["en"]=eng_fr["en"].apply(lambda x: re.sub(" +"," ",x))
eng_fr["fr"]=eng_fr["fr"].apply(lambda x: re.sub(" +"," ",x))

eng_fr

### D) Making a list of all sentences

In [ ]:
X=eng_fr["en"].tolist()
Y=eng_fr["fr"].tolist()

### E) Functions for pre-processing the sentences

 #####  (i) Function to make a corpus from the available sentences



In [ ]:
def to_corpus(sent_list):
  text_corpus=""
  for sentence in sent_list:
    text_corpus+=sentence.lower()+" "
  return text_corpus

##### (ii) Function to train the tokenizer

In [ ]:
def train_tokenizer(file_path):
  tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
  trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
  tokenizer.pre_tokenizer = Whitespace()
  files=[file_path]
  tokenizer.train(files, trainer)
  tokenizer.post_processor = TemplateProcessing(single="[CLS] $A [SEP]", pair="[CLS] $A [SEP] $B:1 [SEP]:1",special_tokens=[("[CLS]", tokenizer.token_to_id("[CLS]")),("[SEP]", tokenizer.token_to_id("[SEP]")),],)
  return tokenizer

##### (iii) Function to form sequence of integers for each sentence

In [ ]:
def sequences(tokenizer,sent_list):
  prepoc_sentences=[]
  for sent in sent_list:
    encoding=tokenizer.encode(sent.lower())
    prepoc_sentences.append(encoding.ids)
  prepoc_sentences = pad_sequences(prepoc_sentences,55, padding='post')
  prepoc_sentences=np.array(prepoc_sentences)
  return prepoc_sentences

##### (iv) Function to form a dictionary of words

In [ ]:
def vocabulary(path_vocab):
    import json
    f=open(path_vocab)
    vocab=json.load(f)
    dict_vocab={}
    for i in vocab["model"]["vocab"]:
        dict_vocab[vocab["model"]["vocab"][i]]= i
    return dict_vocab

### F) Making a corpus out of the sentences

In [ ]:
f1=open("x.txt","w")
f1.write(to_corpus(X))

f2=open("y.txt","w")
f2.write(to_corpus(Y))

2368969

### G) Training and saving the tokenizer with the corpus made

In [ ]:
tokenizer_eng,tokenizer_fr=train_tokenizer("/content/x.txt"),train_tokenizer("/content/y.txt")
tokenizer_eng.save("english_vocab.json",pretty=True)
tokenizer_fr.save("french_vocab.json",pretty=True)

### H) Forming a sequence for each sentence

In [ ]:
prepoc_english_sentences,prepoc_french_sentences=sequences(tokenizer_eng,X),sequences(tokenizer_fr,Y)

### I) Creating a dictionary of words and ids

In [ ]:
dict_eng,dict_fr=vocabulary("english_vocab.json"),vocabulary("french_vocab.json")

#3. Building and Training the se2seq model

### A) Building the model

In [ ]:
def build_model():
    learning_rate=0.003
    model=Sequential()
    model.add(Embedding(30000,256))
    model.add(Bidirectional(GRU(256,return_sequences=True)))  #Bidirectional LSTM would be better
    model.add(Dense(1024, activation='selu'))
    model.add(Dropout(0.1))                                   #Adding an attention layer will also optimise the model
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(Dense(1024, activation='selu'))
    model.add(Dense(30000, activation='softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

### B) Training the model

In [ ]:
tmp_x = pad_sequences(prepoc_french_sentences, maxlen = 55, padding = 'post')


# Train
model = build_model()

model.summary()

model.fit(tmp_x, prepoc_english_sentences, batch_size=64, epochs=20, validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 256)         7680000   
                                                                 
 bidirectional_6 (Bidirecti  (None, None, 512)         789504    
 onal)                                                           
                                                                 
 dense_9 (Dense)             (None, None, 1024)        525312    
                                                                 
 dropout_3 (Dropout)         (None, None, 1024)        0         
                                                                 
 bidirectional_7 (Bidirecti  (None, None, 512)         1969152   
 onal)                                                           
                                                                 
 dense_10 (Dense)            (None, None, 1024)       

#4. Predictions

### A) Function to Convert sequence to sentence

In [8]:
def logits_to_text(logits,dict_lan):
  while logits[-1]==0:
    logits.pop(-1)
  return ' '.join([dict_lan[prediction] for prediction in np.argmax(logits, 1)])

### B) Prediction from training set

In [ ]:
i= 8

print("Prediction:")
test_val=logits_to_text(model.predict(tmp_x[[i]])[0], dict_eng)
print(test_val)
print("\nCorrect Translation:")
actual_val=X[i]
print(X[i])
print("\nOriginal text:")
print(Y[i])

#5. Metrics

In [ ]:
import nltk

hypothesis = test_val.split()
reference = actual_val.split()
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)
